# SETUP

In [105]:
import yfinance as yf
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import plotly.graph_objects as go
from dash import Dash, dcc, html


In [57]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, unix_timestamp, to_date, explode
from pyspark.sql.types import StructType, StructField, DateType, FloatType, StringType, DoubleType

spark = (SparkSession.builder
        .appName("MySparkApp")
        .master("local[*]")
        .config("spark.driver.memory", "4g")
        .config("spark.executor.memory", "4g")
        .getOrCreate()
        )
print(spark.sparkContext.appName)
print(spark.version)

MySparkApp
4.0.0


# DATA REQUEST

In [179]:
# INITIALIZATION
pageSize = 100
startDate = "2025-01-01"
endDate = "2025-08-10"

In [180]:
# HKMA API URLs for yield data
hibor_url = f"https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-daily?pagesize={pageSize}&from={startDate}&to={endDate}"

In [181]:
hibor_response = requests.get(hibor_url).text
json_file_path = "hibor_data.json"
with open(json_file_path, 'w') as f:
    f.write(hibor_response)

In [183]:
hibor_df = spark.read.json(json_file_path)
hibor_df = (hibor_df.select(explode(col("result.records")).alias("a")).select("a.end_of_day", "a.ir_overnight", "a.ir_1w", "a.ir_1m", "a.ir_3m", "a.ir_6m", "a.ir_12m")
        .withColumn("end_of_day", to_date(col("end_of_day"), "yyyy-MM-dd"))
        .withColumn("ir_overnight", col("ir_overnight").cast(FloatType()))
        .withColumn("ir_1w", col("ir_1w").cast(FloatType()))
        .withColumn("ir_1m", col("ir_1m").cast(FloatType()))
        .withColumn("ir_3m", col("ir_3m").cast(FloatType()))
        .withColumn("ir_6m", col("ir_6m").cast(FloatType()))
        .withColumn("ir_12m", col("ir_12m").cast(FloatType()))
        .withColumn('year', F.year(col('end_of_day')))
        .withColumn('month', F.month(col('end_of_day')))
        )
tmp_hibor_df_Month = hibor_df.groupBy("year", "month").agg(
        F.max("end_of_day").alias("end_of_day"),   
)
hibor_df = tmp_hibor_df_Month.join(hibor_df, on = ['end_of_day', 'year', 'month'], how = "left")
hibor_df.show()

+----------+----+-----+------------+-------+-------+-------+-------+-------+
|end_of_day|year|month|ir_overnight|  ir_1w|  ir_1m|  ir_3m|  ir_6m| ir_12m|
+----------+----+-----+------------+-------+-------+-------+-------+-------+
|2025-05-30|2025|    5|     0.03143|0.15566|0.58762|1.32286|2.16417| 2.9125|
|2025-06-30|2025|    6|     0.02839|0.15637|0.72744|1.67958|2.37625|2.96274|
|2025-07-31|2025|    7|     0.18506|0.62381|1.03363| 1.6175|2.26839|2.80185|
|2025-03-31|2025|    3|     3.85476|3.42679| 3.7275|3.88244|3.97417|4.05304|
|2025-04-30|2025|    4|         4.5|4.18155|3.95333| 3.9872|4.02893|4.05446|
+----------+----+-----+------------+-------+-------+-------+-------+-------+



In [188]:
duration_columns = ["ir_overnight", "ir_1w", "ir_1m", "ir_3m", "ir_6m", "ir_12m"]
duration_names = ["overnight", "1w", "1m", "3m", "6m", "12m"]

df_list = []
for i, col_name in enumerate(duration_columns):
    temp_df = hibor_df.select(
        col("end_of_day"),
        lit(duration_names[i]).alias("duration"),
        F.when(col(col_name).isNotNull(), col(col_name)).alias("yield")
    )
    df_list.append(temp_df)
df_melted = df_list[0]
for temp_df in df_list[1:]:
    df_melted = df_melted.union(temp_df)
df_pandas = df_melted.toPandas()
df_pandas["end_of_day"] = pd.to_datetime(df_pandas["end_of_day"])
df_pandas = df_pandas.sort_values("end_of_day", ascending=False)
duration_map = {name: i for i, name in enumerate(duration_names)}
df_pandas["duration_idx"] = df_pandas["duration"].map(duration_map)

In [198]:
df_pandas["end_of_day"] = pd.to_datetime(df_pandas["end_of_day"])
df_pandas = df_pandas.sort_values("end_of_day", ascending=True)
duration_names = ["overnight", "1w", "1m", "3m", "6m", "12m"]


df_pandas

,end_of_day,duration,yield,duration_idx
28,2025-03-31,12m,4.05304,5
23,2025-03-31,6m,3.97417,4
8,2025-03-31,1w,3.42679,1
18,2025-03-31,3m,3.88244,3
13,2025-03-31,1m,3.72750,2
3,2025-03-31,overnight,3.85476,0
29,2025-04-30,12m,4.05446,5
4,2025-04-30,overnight,4.50000,0
24,2025-04-30,6m,4.02893,4
9,2025-04-30,1w,4.18155,1


In [205]:
# 創建 Dash 應用
app = Dash(__name__)

# 為每個月尾日期創建折線圖（不同顏色）
dates = df_pandas["end_of_day"].dt.strftime("%Y-%m-%d").unique()
colors = ['#FF6384', '#36A2EB', '#FFCE56', '#4BC0C0', '#9966FF']  # 為五個月分配顏色
data = []

for i, date in enumerate(dates):
    df_date = df_pandas[df_pandas["end_of_day"].dt.strftime("%Y-%m-%d") == date]
    df_date = df_date.sort_values("duration_idx")  # 確保按 duration_idx 順序
    trace = go.Scatter3d(
        x=df_date["duration_idx"],
        y=df_date["yield"],
        z=df_date["end_of_day"],
        mode='lines+markers',
        name=date,
        line=dict(color=colors[i % len(colors)], width=3),
        marker=dict(size=5, color=colors[i % len(colors)]),
        text=df_date["duration"] + " (" + date + ")",
        hoverinfo='text+y'
    )
    data.append(trace)

# 設置圖表布局
fig = go.Figure(data=data)
fig.update_layout(
    title="3D Yield Curve by Duration and Month-End Date",
    scene=dict(
        xaxis=dict(
            title="Duration",
            tickvals=list(range(len(duration_names))),
            ticktext=duration_names,
            tickangle=45
        ),
        yaxis=dict(
            title="Yield (%)",
            tickformat=".2f"
        ),
        zaxis=dict(
            title="Date",
            tickformat="%Y-%m"
        )
    ),
    width=900,
    height=700,
    showlegend=True,
    margin=dict(l=50, r=50, b=50, t=50)
)

# 定義 Dash 布局
app.layout = html.Div([
    html.H1("3D Yield Curve Dashboard (Month-End Data)"),
    dcc.Graph(id='3d-yield-curve', figure=fig)
])


In [206]:
app.run(mode='jupyter', port=8050)


In [ ]:
spark.stop()